# Στοχαστικές Διεργασίες και Βελτιστοποίηση στη Μηχανική Μάθηση

## 9ο Εργαστήριο - *Radial Basis Function*

- Ονομ/νυμο: Χρήστος Νίκου
- AM: 03400146
- Ιδιότητα: Μεταπτυχιακός φοιτητής Επιστήμης Δεδομένων και Μηχανικής Μάθησης (ΕΔΕΜΜ)
- Ηλεκτρονική Διεύθυνση: christosnikou@mail.ntua.gr / chrisnick92@gmail.com

# Radial Basis Function

Σκοπός της παρούσας άσκησης είναι η πρόβλεψη ως προς το αν ένας/μία πελάτης/ισσα μίας τράπεζας θα κάνει τη χρήση της προθεσμιακής κατάθεσης ή όχι. Αυτή η πρόβλεψη θα εξαχθεί με βάση συγκεκριμένα δημογραφικά χαρακτηριστικά του/της
όπως η ηλικία, η εργασία, η εκπαίδευση, η οικογενειακή κατάσταση κ.λπ.
Για να το επιτύχουμε αυτό θα κάνουμε χρήση μίας εφαρμογής βασισμένη σε RBF. Στο κρυφό επίπεδο των νευρώνων υπολογίζονται τα κέντρα των clusters με τη χρήση του αλγορίθμου k-means.


 

**Βήμα 1ο**: Αρχικά φορτώνουμε τις απαράιτητες βιβλιοθήκες

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import math
from sklearn.metrics import accuracy_score
import os

path = '/content/drive/MyDrive/ColabNotebooks/StochasticLabs/Lab9'

**Βήμα 2ο**: διάβαζουμε το αρχείο bank-full.csv και απομονώνουμε τις στήλες με τα χαρακτηριστικά που θα χρησιμοποιήσουμε (εργασία, αν έχει δανειακές υποχρεώσεις κλπ). Το αρχείο είναι διαθέσιμο [εδώ](https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab9/bank-full.csv) και θα πρέπει να το φορτώσετε στο colab.

In [ ]:
dataset = pd.read_csv(os.path.join(path,'bank-full.csv'),sep=None)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
col_to_use = ['age','balance','day','duration','campaign','pdays','previous']
data = dataset.drop(col_to_use,axis=1)

In [ ]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,may,unknown,no
1,technician,single,secondary,no,yes,no,unknown,may,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,may,unknown,no
3,blue-collar,married,unknown,no,yes,no,unknown,may,unknown,no
4,unknown,single,unknown,no,no,no,unknown,may,unknown,no


In [ ]:
data = data.apply(LabelEncoder().fit_transform)

In [ ]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,4,1,2,0,1,0,2,8,3,0
1,9,2,1,0,1,0,2,8,3,0
2,2,1,1,0,1,1,2,8,3,0
3,1,1,3,0,1,0,2,8,3,0
4,11,2,3,0,0,0,2,8,3,0


In [ ]:
data['job'].unique()

array([ 4,  9,  2,  1, 11,  5,  0,  7,  6, 10,  3,  8])

In [ ]:
dataset['job'].unique()

array(['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'], dtype=object)

In [ ]:
data_rest = dataset[col_to_use]
data_rest.head()

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0


In [ ]:
dataset2 = pd.concat([data_rest,data],axis=1)

In [ ]:
dataset2.head()

,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,58,2143,5,261,1,-1,0,4,1,2,0,1,0,2,8,3,0
1,44,29,5,151,1,-1,0,9,2,1,0,1,0,2,8,3,0
2,33,2,5,76,1,-1,0,2,1,1,0,1,1,2,8,3,0
3,47,1506,5,92,1,-1,0,1,1,3,0,1,0,2,8,3,0
4,33,1,5,198,1,-1,0,11,2,3,0,0,0,2,8,3,0


**Βήμα 3ο**: Σπάμε στο dataset σε train και test dataset και κάνουμε scale με τη χρήση της συνάρτησης StandardScaler()

In [ ]:
X= dataset2.drop('y',axis=1)
y= dataset2['y']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.33, random_state= 4)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

array([[ 3.39736083, -0.45852779,  0.14087303, ..., -0.71332324,
        -1.50513854,  0.44672656],
       [-0.27605947,  2.43941798,  0.50067953, ...,  0.40010942,
        -1.83733874, -2.58685457],
       [ 0.66584317, -0.13326522, -0.93854645, ..., -0.71332324,
         1.48466323, -0.56446715],
       ...,
       [-0.74701079, -0.22619739,  0.62061503, ..., -0.71332324,
         1.15246303,  0.44672656],
       [ 1.60774581, -0.42275894, -1.53822394, ...,  1.51354209,
         0.15586244,  0.44672656],
       [ 0.28908211, -0.13627382,  1.58009901, ..., -0.71332324,
        -0.17633776,  0.44672656]])

**Βήμα 4ο**: Ορίζουμε τα κέντρα των νευρώνων με τη χρήση του k-means. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters.

In [ ]:
K_cent= 8
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

In [ ]:
max=0 
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)

Βήμα 5ο: Δημιουργούμε τον πίνακα σχέσεων F (γραμμικός συνδυασμός των Κ συναρτήσεων βάσης φj(x).

In [ ]:
shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

In [ ]:
for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 6ο**: Βρίσκουμε τον πίνακα βάρους W για την εκπαίδευση του δικτύου

In [ ]:
FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

**Βήμα 7ο**: Ρύθμιση του πίνακα F για το test dataset

In [ ]:
row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 8ο**: Ακρίβεια πρόβλεψης στο test dataset

In [ ]:
prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8876675603217158


## Ερωτήσεις

**Ερώτηση 1**: Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF

**Ερώτηση 2**:Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

**Ερώτηση 3**: Επιχειρήστε να αλλάξετε την τιμή του k σε 10 και στη συνέχεια σε 50. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;

### Ερώτηση 1

>*Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF*

Ένα νευρωνικό δίκτυο RBF αποτελείται συνήθως από τρία layer, το layer εισόδου (input), το κρυφό layer (hidden) μαζί με μια μη γραμμική συνάρτηση ενεργοποίησης τύπου RBF και ένα γραμμικό layer εξόδου. Πιο συγκεκριμένα, αν συμβολίσουμε με $x\in \mathbb{R}^n$ το διάνυσμα των χαρακτηριστικών εισόδου, τότε ένα νευρωνικό δίκτυο RBF είναι ουσιαστικά μια συνάρτηση $\phi:\mathbb{R}^n\to \mathbb{R}$ για την οποία<br>

$$\tag{1}\phi(x) = \sum_{i=1}^{N}\alpha_i\rho\bigl(| | x-c_i | |\bigr),$$<br>

όπου $N$ είναι ο αριθμός των νευρώνων στο κρυφό επίπεδο, $c_i$ είναι διάνυσμα που εκφράζει το «κέντρο» του νευρώνα $i$ στο κρυφό επίπεδο και $\alpha_i$ είναι το βάρος του νευρώνα $i$ στην έξοδο του νευρώνα. Συναρτήσεις των οποίων οι τιμές εξαρτώνται από ένα διάνυσμα «κέντρου» είναι ακτινικά συμμετρικές γύρω απ' αυτό το σημείο και έτσι αυτές οι συναρτήσεις είναι γνωστές ως *Radial Basis Functions* στην αγγλική ορολογία. Η νόρμα που εμφανίζεται στην $(1)$ είναι συνήθως η Ευκλείδεια νόρμα και η πιο συνηθισμένη συνάρτηση *RBF* που χρησιμοποιείται είναι η Γκαουσιαννή που δίνεται μέσω της<br>

$$\rho\bigl(| | x-c_i| |\bigr)=\exp\biggl(-\beta_i| | x-c_i| |^2\biggr).$$<br>

Η εκπαίδευση ενός νευρωνικού δικτύου *RBF* γίνεται σε δύο βήματα. Στο πρώτο βήμα επιλέγονται τα διανύσματα $c_i$ που αντιστοιχούν στα κέντρα των *RBF* συναρτήσεων. Η επιλογή των κέντρων μπορεί να γίνει με διάφορους τρόπους. Για παράδειγμα, μπορεί να επιλεχτούν τυχαία κάποια κέντρα απ' το δείγμα εκπαίδευσης ή μπορεί να προσδιορισθούν μέσω του αλγορίθμου *K-Means* όπως γίνεται και στο παρόν Notebook. Στο δεύτερο βήμα υπολογίζονται τα βάρη $\alpha_i$ που αντιστοιχούν στους νευρώνες του κρυφού επιπέδου.

### Ερώτηση 2

>*Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;*

Ένας τρόπος υπολογισμού των βαρών $\alpha_i$ στο κρυφό layer είναι με επίλυση του γραμμικού συστήματος<br>

$$\phi(x_i) = \sum_{j=1}^{N}\alpha_j\rho\biggl(| | x_i - c_j| | \biggr)=d_i,$$
<br>

όπου $\{(x_i,d_i)\}_{i=1}^{M}$ είναι το σύνολο εκπαίδευσης μαζί με τις ετικέτες $d_i$. Σε αυτή την περίπτωση οι σχέσεις $\phi(x_i)=d_i,i=1,2\dots M$ ορίζουν μια υπερ-επιφάνεια δυαδικού διαχωρισμού κλάσεων για το δείγμα μάθησης. Ένας άλλος τρόπος με τη μέθοδο των ελαχίστων τετραγώνων. Δηλαδή, σε αυτή την περίπτωση σκοπός είναι η ελαχιστοποίηση της ποσότητας<br>

$$\sum_{i=1}^{M}\biggl(d_i-\sum_{j=1}^{M}\alpha_j \rho\biggl(| | x_i - c_j| | \biggr)\biggr)^2,$$<br>

ως προς τις μεταβλητές των βαρών $\alpha_j$. Ο τρόπος ελαχιστοποίησης μπορεί να γίνει είτε με μεθόδους κατάβασης είτε να λυθεί αναλυτικά με μεθόδους γραμμικής παλινδρόμησης.

### Ερώτηση 3

>*Επιχειρήστε να αλλάξετε την τιμή του k σε 10 και στη συνέχεια σε 50. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;*

Παρακάτω βλέπουμε τα αποτέλεσματα για την τιμή του $k=10$.

In [ ]:
K_cent= 10
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

max=0 
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)


shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)


for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))


FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)


row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8876675603217158


Και τέλος και για την περίπτωση που $K=50$.

In [ ]:
K_cent= 50
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

max=0 
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)


shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)


for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))


FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)


row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8940348525469168


Στον παρακάτω πίνακα συνοψίζουμε την ακρίβεια για κάθε μια απ' τις παραπάνω τρεις περιπτώσεις.


| **Αριθμός Κέντρων** | **Ακρίβεια**|
|:---------------|:-----------------|
| **8** |       88%      |
| **10** |       88%     |
| **50**      |       89%     |


Αυτό που παρατηρούμε είναι ότι στην περίπτωση που έχουμε $50$ κέντρα αυξάνεται η ακρίβεια κατά 1%. Στις άλλες δύο περιπτώσεις δεν παρατηρούμε κάποια ουσιαστική διαφορά.